## Linear Models - Partial Least Squares Regression and the LASSO
#### a.k.a. Review vs. Revenue

In [ ]:
# load and show dataset
import pandas as pd
import datetime
import numpy as np
import seaborn as sns
import sklearn
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.impute import SimpleImputer
from sklearn.cross_decomposition import PLSRegression
from sklearn.linear_model import Lasso, LinearRegression
from sklearn.dummy import DummyRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats
%matplotlib inline
set_matplotlib_formats('svg')
sns.set_style("darkgrid")

In this notebook we try to predict the revenue of a movie based on its reviews using the __Rotten Tomato Movie Review Dataset__ (http://users.stat.ufl.edu/~winner/datasets.html)

Variables:
- Rank   
- AllPos    (# Positive Reviews among all critics)
- AllNeg    (# Negative Reviews among all critics)
- TopPos    (# Positive Reviews Among Top critics)
- TopNeg    (# Negative Reviews Among Top critics)
- PropAllPos  (Proportion Positive Reviews among all critics)
- PropTopPos  (Proportion Positive Reviews among top critics)
- Movie
- Release     (Date of Release)
- Rev_10M     (revenue, in $10Ms)
- year        (year of Release)
- AllFresh    (1 if PropAllPos >= 0.60, 0 otherwise)

### 1. Data Inspection and Preprocessing
When we load the data and examine the types, we notice that the release date is encoded as a string. We are not dealing with a Time Series problem here, so we cannot easily use the date in our model. Therefore we split the date into day, month and year. In addition, we calculate a kind of relative date by setting each release in relation to the oldest release. We also have to fix that `PropTopPos` is an object. 

In [ ]:
data = pd.read_csv('box_office_rt.csv')
data['Release'] = pd.to_datetime(data.Release) #convert the string object Release to datetime datatype
data['month'] = data.Release.dt.month
data['day'] = data.Release.dt.day
data['rel_release'] = data.Release.subtract(data.Release.min()) 
data['rel_release'] = data.rel_release.dt.total_seconds()/(3600*24) #release relative to the 1st entry (in days)
print(data.PropTopPos[(data.TopPos==0) & (data.TopNeg==0)]) #print wrong/missing values
data['PropTopPos'] = pd.to_numeric(data['PropTopPos'], errors='coerce') #convert variable to numeric - "#DIV/0" will be set to NaN
print(data.PropTopPos[(data.TopPos==0) & (data.TopNeg==0)]) #print NaN values

In [ ]:
data.isnull().sum()[(data.isnull().sum() > 0)] #check for missing values

You can use the line of code below to perfrom scatter plots of variables which are of your interest.

In [ ]:
sns.scatterplot(x='Rank', y='Rev_10M',data=data)

In [ ]:
X = data.drop(columns=['Rev_10M','Rank', 'Movie','Release'])
y = data.Rev_10M
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.33, random_state=23)

Even though we see a good correlation of `Rank` with `Rev_10m` for an unknown film, we would not yet have the information about the `Rank` based on the revenue, since we would have to calculate it first. Therefore, we will not use this variable. We will also drop the variable `Movie` and the datetime-formatted `Release`.

You can inspect the correlation matrix below. (Code snippet taken from [seaborn docs](https://seaborn.pydata.org/examples/many_pairwise_correlations.html))

In [ ]:
# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(data.loc[X_train.index].corr(), dtype=bool))
# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))
# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)
# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(data.loc[X_train.index].corr(), mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

### 2. Partial Least Squares Regression
We encoutered 4 missing values which we have to take care of. The easiest and simplest solution is to intergrate a `SimpleImputer` in the Pipeline. It fills the missing values with the column mean. As an initial model - we fit a PLS regression on the training set using 3 components.

In [ ]:
xscaler = StandardScaler()
yscaler = StandardScaler()
pipe = Pipeline([('imputer', SimpleImputer()),('scaler', xscaler), ('plsr', PLSRegression(n_components=3))])
model = TransformedTargetRegressor(regressor=pipe, transformer=yscaler)

In [ ]:
model.fit(X_train,y_train)

We add the X-scores (the "new" latent X-variables) to the dataframe. 

In [ ]:
T = model.regressor_['plsr'].x_scores_
data_pls = pd.DataFrame(columns=X_train.columns,data=xscaler.fit_transform(X_train))
data_pls[['T1', 'T2', 'T3']] = pd.DataFrame(T)
data_pls.head()

Now we observe the scores plot of the first and second component, we integrate the information about the revenue by color. The outcome is a bit indifferent. We can not see clear clusters and we can also not see a clear relationship between the revenue and the X-scores. We can see that the highest grossing movies (very dark color) are not lying close together in the T1-T2-space.

In [ ]:
plt.figure()
sns.scatterplot(x='T1',y='T2',hue=(yscaler.fit_transform(y_train.values.reshape(-1,1))).ravel(), data=data_pls)
plt.show()

The conterpart to scores plots in PCA/PLSR domain, are the so called loadings plots. Also scatter plots, but now we plot the loadings which are used to transform the oroginal variables to their new latent space. We can use loading plots to interpret which features contrubute to the components. The problem with loading plots is the scaling. Interpretation is easier by using so called correlation loadings. The correlation loadings are bounded between -1 and 1 and thus easier to compare.

In [ ]:
sns.scatterplot(x = model.regressor_['plsr'].x_loadings_[:,0],y=model.regressor_['plsr'].x_loadings_[:,1])
plt.xlabel('P1')
plt.ylabel('P2')

The correlation loadings can be calculeted directly from the loadings or by computing the pearson correlation coefficient between the oroginal features and the scores. We will use the latter approach, as pandas gives us the pairwise correlations with a single command.

In [ ]:
corL = data_pls.corr()
corL.drop(corL.columns[:-3],axis=1,inplace=True)
corL.drop(corL.index[-3:],axis=0,inplace=True)
corL

Now we can again go for the correlation loadings plot. We also add the circles of 50% variance and 100% variance.

In [ ]:
sns.scatterplot(x='T1', y='T2', data=corL)
plt.plot(np.sin(np.linspace(0,2*np.pi,100)),np.cos(np.linspace(0,2*np.pi,100)),'--',color='.5')
plt.plot(np.sqrt(0.5)*np.sin(np.linspace(0,2*np.pi,100)),np.sqrt(.5)*np.cos(np.linspace(0,2*np.pi,100)), '--', color='.8')
for kk,nme in enumerate(corL.index):
     plt.annotate(nme,(corL.T1.values[kk]+.01,corL.T2.values[kk]))
plt.ylim(-1.1,1.1)
plt.xlim(-1.1,1.1)

We can see that `rel_release` and `TopPos` and  `AllPos` are pretty much exactly on the direction of the first component. Whereas `rel_release` is only  weakly correlated (i.e., close to the origin), the other two features contribute very strongly (positively) to the first component. 
`month` is slightly positively associated with the second component, whereas `TopNeg` and `AllNeg` are negatively correlated with it. 
The percentage features, as well as `AllFresh` put one are around 45 degrees, the release year at -45 degrees.

In [ ]:
plt.figure()
sns.scatterplot(x='T1', y='T3', data=corL)
plt.plot(np.sin(np.linspace(0,2*np.pi,100)),np.cos(np.linspace(0,2*np.pi,100)),'--',color='.5')
plt.plot(np.sqrt(0.5)*np.sin(np.linspace(0,2*np.pi,100)),np.sqrt(.5)*np.cos(np.linspace(0,2*np.pi,100)), '--', color='.8')
for kk,nme in enumerate(corL.index):
     plt.annotate(nme,(corL.T1.values[kk]+.01,corL.T3.values[kk]))
plt.ylim(-1.1,1.1)
plt.xlim(-1.1,1.1)
plt.show()

We do not see any excessively strong correlations for the third component. We that `day` is nagatively correlated and a slightly stronger correlation is observed with `rel_release`.

In [ ]:
plt.figure()
sns.scatterplot(x='T2', y='T3', data=corL)
plt.plot(np.sin(np.linspace(0,2*np.pi,100)),np.cos(np.linspace(0,2*np.pi,100)),'--',color='.5')
plt.plot(np.sqrt(0.5)*np.sin(np.linspace(0,2*np.pi,100)),np.sqrt(.5)*np.cos(np.linspace(0,2*np.pi,100)), '--', color='.8')
for kk,nme in enumerate(corL.index):
     plt.annotate(nme,(corL.T2.values[kk]+.01,corL.T3.values[kk]))
plt.ylim(-1.1,1.1)
plt.xlim(-1.1,1.1)
plt.show()

So far, we have worked with an initial estimate of the number of compnents to address the specifics of PLS (Scores and loadings plot). In many textbooks only 2 components are often used for visualization and interpretation. However, we will of course perform a grid search to determine the number of components that gives us the smallest RMSE. The nice thing about PLSR is that the parameter space is discrete and constrained.

In [ ]:
cval = KFold(10, random_state=60, shuffle=True)
pipe1 = Pipeline([('imputer', SimpleImputer()),('scaler', xscaler), ('plsr', PLSRegression(n_components=3))])
model1 = TransformedTargetRegressor(regressor=pipe1, transformer=yscaler)
param_grid1 = dict()
param_grid1['regressor__plsr__n_components'] = np.arange(1,X_train.shape[1])
search1 = GridSearchCV(model1, param_grid1, scoring=['neg_root_mean_squared_error','r2'], n_jobs=-1, cv=cval, refit='neg_root_mean_squared_error', return_train_score=True)

In [ ]:
search1.fit(X_train,y_train)
results1 = pd.DataFrame(search1.cv_results_)
print("Best parameter RMSE=%0.3f):" % (-search1.best_score_))
print(search1.best_params_)

plt.figure()
plt.errorbar(results1.param_regressor__plsr__n_components, -results1.mean_train_neg_root_mean_squared_error, yerr=results1.std_train_neg_root_mean_squared_error, label='Train')
plt.errorbar(results1.param_regressor__plsr__n_components, -results1.mean_test_neg_root_mean_squared_error, yerr=results1.std_test_neg_root_mean_squared_error, label = 'Test (CV)')
plt.legend()
plt.xlabel("# of components")
plt.ylabel("RMSE")
plt.title("PLSR")
plt.show()

We can observe a quite high variance for each of the components and we can also see that more components than initially guessed are neccessary for the lowest RMSE in out setting. It is interesting to observe that for more than 5 coponents we run into overfitting (decreasing RMSE (train) and increasing RMSE (test)). Furthermore we have to state that the RMSE is quite high.

### 3. Lasso Regresison
As a representative of the regularizing models we choose Lasso. The difference to ridge regression is the norm of the coefficient vector. In this case, the $l_1$ norm is used, which favors a sparse coefficient vector. This implicitly performs a feature selection - non-relevant features are set to 0. However, if there are several strongly correlated (but "important") variables, one may be selected and the others set to zero. The parameter we want to tune via gridsearch is the weight for your constraint for the $l_1$ norm. 

In [ ]:
pipe2 = Pipeline([('imputer', SimpleImputer()),('scaler', xscaler), ('lasso', Lasso())])
model2 = TransformedTargetRegressor(regressor=pipe2, transformer=yscaler)
param_grid2 = {'regressor__lasso__alpha': [1e-9,1e-3,5e-3,8e-3, 1e-2,2e-2,3e-2, 8e-2,1e-1,1]}
search2 = GridSearchCV(model2, param_grid2, scoring=['neg_root_mean_squared_error','r2'], n_jobs=-1, cv=cval, refit='neg_root_mean_squared_error', return_train_score=True)

In [ ]:
search2.fit(X_train, y_train)
print("Best parameter RMSE=%0.3f):" % (-search2.best_score_))
print(search2.best_params_)

plt.figure()
plt.errorbar(param_grid2['regressor__lasso__alpha'],-search2.cv_results_['mean_train_neg_root_mean_squared_error'],yerr=search2.cv_results_['mean_train_neg_root_mean_squared_error'],label='Train')
plt.errorbar(param_grid2['regressor__lasso__alpha'],-search2.cv_results_['mean_test_neg_root_mean_squared_error'],yerr=search2.cv_results_['mean_test_neg_root_mean_squared_error'],label='Test')
plt.gca().set_xscale('log')
plt.legend()
plt.xlabel("alpha")
plt.ylabel("RMSE")
plt.title("Lasso")
plt.show()

We see that moderate regularization leads to the best results. When alpha approaches zero, the model approaches a "normal" linear regression. For larger alpha values, the RMSE deteriorates significantly. The error is in a similar range as for PLSR.

### 4. Final Performance
Finally we will asses the performance on the held out test set.

In [ ]:
best_estimator1 = search1.best_estimator_
y_pred_train1 = best_estimator1.predict(X_train)
y_pred_test1 = best_estimator1.predict(X_test)
best_estimator2 = search2.best_estimator_
y_pred_train2 = best_estimator2.predict(X_train)
y_pred_test2 = best_estimator2.predict(X_test)


minlim = np.min([y_test.min(), np.min(y_pred_test1), np.min(y_pred_test2)])-1
maxlim = np.max([y_test.max(), np.max(y_pred_test1), np.max(y_pred_test2)])+1
# predicted/actual plot for test set
ax = sns.jointplot(x=y_test.ravel(),y=y_pred_test1.ravel(), height=5,xlim=(minlim,maxlim),ylim=(minlim,maxlim))
ax.ax_joint.set_xlabel('observed player performance')
ax.ax_joint.set_ylabel('predicted player performance')
plt.show()


# predicted/actual plot for test set
ax = sns.jointplot(x=y_test.ravel(),y=y_pred_test2.ravel(), height=5,xlim=(minlim,maxlim),ylim=(minlim,maxlim))
ax.ax_joint.set_xlabel('observed player performance')
ax.ax_joint.set_ylabel('predicted player performance')
plt.show()

Unfortuenately the diagnostic plots above look not very promising - so, what is it at least good for? Let's compare our models to the simplest linear model, aka linear regression as well as a dummy estimator. The latter serves as a worst case estimate. 

In [ ]:
### lr baseline
pipelr = Pipeline([('imputer', SimpleImputer()),('scaler', xscaler), ('lr', LinearRegression())])
modellr = TransformedTargetRegressor(regressor=pipelr, transformer=yscaler)
modellr.fit(X_train,y_train)

### dummy worst case
pipedm = Pipeline([('imputer', SimpleImputer()),('scaler', xscaler), ('dm', DummyRegressor())])
modeldm = TransformedTargetRegressor(regressor=pipedm, transformer=yscaler)
modeldm.fit(X_train,y_train)


est = {'PLSR': best_estimator1, 'Lasso': best_estimator2, 'LinReg': modellr, 'Dummy': modeldm}

fin_res = pd.DataFrame({'model': ['PLSR', 'Lasso','LinReg', 'Dummy'], 'RMSE train': np.empty(4), 'R2 train': np.empty(4), 'RMSE test': np.empty(4), 'R2 test': np.empty(4)}).set_index('model')

for m in ['PLSR', 'Lasso', 'LinReg','Dummy']:
    y_pred_train = est[m].predict(X_train)
    y_pred_test = est[m].predict(X_test)
    fin_res.at[m,'RMSE train'] = np.sqrt(mean_squared_error(y_train,y_pred_train))
    fin_res.at[m,'R2 train'] = r2_score(y_train,y_pred_train)
    fin_res.at[m,'RMSE test'] = np.sqrt(mean_squared_error(y_test,y_pred_test))
    fin_res.at[m,'R2 test'] = r2_score(y_test,y_pred_test)
fin_res

The Lasso outperforms all other approaches, but we can not see huge differences. Even if we would build no model at all (=Dummy) we achieve a RMSE of approx. 70 million dollars. In this case we would have a hard time to sell this model. It seems that we are missing some important predictors. You can at least check if a more flexible, non-linear model would perform better.

As a last step, we compare the coefficients from different models. We should also be able to see that the LASSO leads to coefficients equal to 0. The coefficients will be ranked according to their magnitude. 

In [ ]:
f,axs = plt.subplots(1,3, figsize=(8,3))
coef1 = pd.Series(best_estimator1.regressor_['plsr'].coef_.ravel(), index = X_train.columns).sort_values(key=lambda x: np.abs(x))
coef2 = pd.Series(best_estimator2.regressor_['lasso'].coef_, index = X_train.columns).sort_values(key=lambda x: np.abs(x))
coef3 = pd.Series(modellr.regressor_['lr'].coef_, index = X_train.columns).sort_values(key=lambda x: np.abs(x))

coef1.plot(kind = "barh",ax = axs[0])
axs[0].set_title("Coefficients PLSR")
axs[0].set_xlim(-.4,1)

coef2.plot(kind = "barh",ax = axs[1])
axs[1].set_title("Coefficients Lasso")
axs[1].set_xlim(-.4,1)

coef3.plot(kind = "barh",ax = axs[2])
axs[2].set_title("Coefficients LinReg")
axs[2].set_xlim(-.4,1)

plt.tight_layout()